# Binaural synthesis
Humans can localize sound events about their perceived distance and their angular position in space.
Sound waves are altered due to reflections, diffraction, and resonances caused by the presence of a human body, head, shoulders, torso, and the fine structure of the ear formed by pinna, cavum conchae, etc.
All these effects, which in its assembly are evaluated by the human brain to localize a source or to get other spatial information, are integrated into binaural signals.
If the binaural signal is reproduced perfectly at the eardrums (the human microphones), there is no chance to acoustically distinguish the virtual source or environment from the real sound field.
With binaural synthesis, a filtering approach with special filters, an acoustic sound source represented by a mono-signal can be virtually placed at arbitrary space positions.

## HRTF dataset

A valid way to describe all linear sound transformations caused by torso, head, and pinna is using so-called “head-related transfer functions” (HRTFs).
For each direction of sound incidence from a sound source to a human receiver, two transfer functions exist (one for the left and one for the right ear), combined into a two-channel HRTF in the frequency domain.
Combining all directions into a single database is commonly called an HRTF dataset.
The time domain version of HRTFs are also known as "head-related impulse responses" (HRIRs).

In [ ]:
%matplotlib inline
# import the required packages
import pyfar as pf
import sofar as sf
import numpy as np
import matplotlib.pyplot as plt
import os

files = pf.signals.files._load_files('head_related_impulse_responses')
hrir_file = os.path.join(pf.signals.files.file_dir, files[0])

# load a SOFA file containing the HRIR dataset
hrirs, sources, _ = pf.io.read_sofa(hrir_file)


Here we are loading the included HRIR dataset from the FABIAN dummy head by [Brinkmann _et al._](https://depositonce.tu-berlin.de/items/3b423df7-a764-4ce1-9065-4e6034bba759).
`pyfar` includes a [method to load specific HRIRs from the dataset](https://pyfar.readthedocs.io/en/stable/modules/pyfar.signals.files.html#pyfar.signals.files.head_related_impulse_responses) but the example shown here is the general approach for loading a SOFA file.

First, we will plot all possible source locations that are contained in the dataset.

In [ ]:
sources.show()

We can see that the included dataset only contains a limited number of source positions - namely, the horizontal plane and the median plane.
In practice, an HRIR dataset usually contains more source positions covering the whole sphere around the listener.

Using the `pyfar` coordinates, <!-- LINK TODO --> a specific direction can be selected from the dataset.

In [ ]:
# define the direction of the desired sound source; here, we choose the right direction
elevation = 0
azimuth = -90

# convert to radians
elevation = elevation * np.pi / 180
azimuth = azimuth * np.pi / 180

desired_direction = pf.Coordinates.from_spherical_elevation(
    azimuth, elevation, 2
)

index, _ = sources.find_nearest(desired_direction)

sources.show(index)

In the plot, the selected direction is marked with a red dot.

With an HRIR selected, we can plot it in the time and frequency domain.

In [ ]:
ax = pf.plot.time_freq(hrirs[index], label=["Left ear", "Right ear"])
ax[0].legend()
ax[1].legend()

## Creating a binaural synthesis

The binaural synthesis is done by convolving a mono, anechoic signal with an HRIR.
The following code shows how to do this with `pyfar`.
We will create a function for this to reuse it later.
This function gets the desired azimuth direction and selects the corresponding HRIR from the dataset.
Then, it convolves the input signal with the HRIR and returns the binaural signal.
As an additional functionality, this function also allows a gain to be applied to the binaural signal.

In [ ]:
def convolve_with_hrir(azimuth, audio, hrirs, gain=1.0):
    # convert to radians
    azimuth = azimuth * np.pi / 180

    # define the direction of the desired sound source
    desired_direction = pf.Coordinates.from_spherical_elevation(
        azimuth, 0, 2
    )

    # find the nearest HRIRs to the desired direction
    index, _ = sources.find_nearest(desired_direction)

    # apply the HRIRs to the audio signal
    output_audio = pf.dsp.convolve(audio, hrirs[index])
    output_audio = 10 ** (gain / 20) * pf.dsp.normalize(output_audio)

    return output_audio

We will use one of the example audio files included in `pyfar` as an input signal.

The following code loads the audio signal, plots the time domain and allows you to listen to it.

In [ ]:
from IPython.display import Audio

castanets = pf.signals.files.castanets()

pf.plot.time(castanets)

Audio(castanets.time, rate=castanets.sampling_rate)

Next, we will create a binaural synthesis from this signal.

In [ ]:
binaural_audio = convolve_with_hrir(-90, castanets, hrirs)

Audio(binaural_audio.time, rate=binaural_audio.sampling_rate)

## Interactive example

Last but not least, we will create an interactive example that allows you to select a source position and listen to the binaural synthesis.
In addition, we will also use two signals to demonstrate the effect of the binaural synthesis.

In [ ]:
# from ipywidgets import (
#     GridspecLayout,
#     Button,
#     Layout,
#     IntSlider,
#     interactive_output,
# )

# guitar = pf.signals.files.guitar(44100)
# guitar.time = guitar.time[:,:castanets.n_samples]


# def interactive_demo(castanets_azimuth, castanets_gain, guitar_azimuth, guitar_gain):
#     castanets_audio = convolve_with_hrir(castanets_azimuth, castanets, hrirs, castanets_gain)
#     guitar_audio = convolve_with_hrir(guitar_azimuth, guitar, hrirs, guitar_gain)

#     mixed_audio = pf.classes.audio.add((castanets_audio, guitar_audio), domain="time")

#     mixed_audio = pf.dsp.normalize(mixed_audio)

#     display(Audio(mixed_audio.time, rate=mixed_audio.sampling_rate, normalize=False))

# castanets_btn = Button(
#     description="Castanets",
#     button_style="success",
#     layout=Layout(height="auto", width="auto"),
# )
# guitar_btn = Button(
#     description="Guitar",
#     button_style="success",
#     layout=Layout(height="auto", width="auto"),
# )
# castanets_az_sl = IntSlider(
#     value=90,
#     min=-180,
#     max=180,
#     step=5,
#     description="Azimuth [deg]",
#     continuous_update=False,
#     layout=Layout(height="auto", width="auto"),
# )
# castanets_gain_sl = IntSlider(
#     value=0,
#     min=-50,
#     max=0,
#     step=1,
#     description="Gain [dB]",
#     continuous_update=False,
#     layout=Layout(height="auto", width="auto"),
# )
# guitar_az_sl = IntSlider(
#     value=-90,
#     min=-180,
#     max=180,
#     step=5,
#     description="Azimuth [deg]",
#     continuous_update=False,
#     layout=Layout(height="auto", width="auto"),
# )
# guitar_gain_sl = IntSlider(
#     value=0,
#     min=-50,
#     max=0,
#     step=1,
#     description="Gain [dB]",
#     continuous_update=False,
#     layout=Layout(height="auto", width="auto"),
# )

# grid = GridspecLayout(3, 2, height="200px")
# grid[0, 0] = castanets_btn
# grid[1, 0] = castanets_az_sl
# grid[2, 0] = castanets_gain_sl
# grid[0, 1] = guitar_btn
# grid[1, 1] = guitar_az_sl
# grid[2, 1] = guitar_gain_sl

# out = interactive_output(
#     interactive_demo,
#     {
#         "castanets_azimuth": castanets_az_sl,
#         "castanets_gain": castanets_gain_sl,
#         "guitar_azimuth": guitar_az_sl,
#         "guitar_gain": guitar_gain_sl,
#     },
# )

# display(grid, out)